In [3]:
#3.5.7
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print(len(train_data))
print(len(test_data))

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics='accuracy')

#3.5.4
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

#3.18
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=4,
                    batch_size=512,
                    validation_data=(x_val, y_val))

results = model.evaluate(x_test, one_hot_test_labels)


8982
2246
Epoch 1/4
16/16 [==============================] - 1s 27ms/step - loss: 2.6704 - accuracy: 0.5342 - val_loss: 1.7643 - val_accuracy: 0.6430
Epoch 2/4
16/16 [==============================] - 0s 16ms/step - loss: 1.4360 - accuracy: 0.7047 - val_loss: 1.3307 - val_accuracy: 0.7160
Epoch 3/4
16/16 [==============================] - 0s 17ms/step - loss: 1.0599 - accuracy: 0.7736 - val_loss: 1.1525 - val_accuracy: 0.7580
Epoch 4/4
71/71 [==============================] - 0s 1ms/step - loss: 1.1390 - accuracy: 0.7538


In [5]:
#3.5.7
print(history.history['accuracy'][-1])
print(history.history['val_accuracy'][-1])

predictions = model.predict(x_test)
print(predictions[0])
print(np.sum(predictions[0]))
print(np.argmax(predictions[0]))

0.8184665441513062
0.7739999890327454
[5.89510193e-04 8.36966443e-04 9.37194389e-04 9.40381706e-01
 3.20718363e-02 1.01624624e-04 7.85640732e-04 2.38886678e-05
 6.55315910e-03 7.33064197e-04 7.74371962e-04 1.33823662e-04
 5.68025513e-04 2.59679393e-03 4.87747238e-05 3.08895069e-05
 6.80126483e-03 4.85670840e-04 4.55660687e-04 4.09621076e-04
 6.07093214e-04 1.09794433e-04 1.14826355e-04 2.07955993e-04
 3.08110553e-04 3.96475574e-04 1.17117343e-04 3.26676786e-06
 7.18885058e-05 2.45234492e-04 2.92380544e-04 6.97422147e-05
 2.75340659e-04 3.00060510e-05 4.77046968e-04 2.79903306e-05
 7.11070781e-04 1.73614353e-05 1.01479738e-04 1.52880835e-04
 2.88639985e-05 5.89458723e-05 1.97843146e-05 8.70600416e-05
 2.92686036e-06 1.45816492e-04]
0.9999999
3
